In [32]:
## Lagrangian for UFCLP

import pandas as pd
import numpy as np
import os
import Dijkstra_v2 as dijk 


#지난 과제인 다익스트라 코드 활용하여 각 노드에서 노드로 가는 최단거리 테이블 생성 d(i,j) 과정 (nxn matrix)
def make_dataframe_dijk(graph_file):    
    dist_df = pd.DataFrame()
    graph,node_number = dijk.read_graph(graph_file)   # 그래프 읽어오기//텍스트 형식의 인접 리스트 넣어주면 그래프와 노드 수 가져옴      
    for start in range(node_number):                          # 최단거리 nxn matrix 가져오기.
        dist_df[start] = dijk.find_shortest_path(graph,start)      # 다익스트라 코드 재활용하여 반복함.
    dist_df.columns = [chr(i+65) for i in np.array(dist_df.columns)]  # 노드명 숫자로 되어있는데 알파벳으로 변환
    dist_df.index = dist_df.columns
    return dist_df, node_number

 # sigma h_i * d_ij table 생성하는 함수
def make_dataframe_h_d(dist_df,node_df):             
    # print(dist_df)
    hd_df = pd.DataFrame(index=dist_df.index, columns = dist_df.columns)  #빈 데이터프레임 생성.
    for i in dist_df.columns:
        hd_df.loc[i] = np.array(dist_df[i])*np.array(node_df).ravel()    # node_df의 shape가 1*n이라 일렬로 펴줘야 해서 ravel() 사용
    hd_df = hd_df.T   # 출력해보니까 뒤집어져 있었다. Transpose 해주자.
    sum_array = np.array(hd_df.sum())
    return hd_df,sum_array

#input file 정의
graph_file = './graph_figure7-4.txt'     #그래프 TXT파일 위치 
dist_df,node_number = make_dataframe_dijk(graph_file)   #각 노드 사이의 최단거리 테이블 생성, 및 노드 수 까지 같이 가져옴

df = pd.read_excel('./UFCLP_nodeInfoTable.xlsx', index_col = 0)   # 노드별 수요, 초기건립비용 저장 테이블 불러오기
demand_df = pd.DataFrame(df['demand'],index=df.index)  # 노드별 수요 데이터
# demand_avg = demand_df

cost_df = pd.DataFrame(df['fixed_cost'],index=df.index) # 노드별 초기건립비용 데이터

# original_dist_df = dist_df   #각 노드 사이 최단거리 테이블은 반복마다 새로 정의되지만  원본도 하나 필요함
original_hd_df,original_sum_array = make_dataframe_h_d(dist_df,demand_df) # 추후 활용


dist_df = dist_df.append(dist_df.mean(), ignore_index=True)
dist_df

,A,B,C,D,E,F,G,H,I,J,K,L
0,0.000000,15.000000,37.0,55.000000,24.0,60.000000,18.000000,33.000000,48.00,40.0,58.00,67.00
1,15.000000,0.000000,22.0,40.000000,38.0,52.000000,33.000000,48.000000,42.00,55.0,61.00,61.00
2,37.000000,22.000000,0.0,18.000000,16.0,30.000000,41.000000,28.000000,20.00,58.0,39.00,39.00
3,55.000000,40.000000,18.0,0.000000,34.0,12.000000,59.000000,46.000000,24.00,62.0,43.00,34.00
4,24.000000,38.000000,16.0,34.000000,0.0,36.000000,25.000000,12.000000,24.00,47.0,37.00,43.00
5,60.000000,52.000000,30.0,12.000000,36.0,0.000000,57.000000,42.000000,12.00,50.0,31.00,22.00
6,11.000000,26.000000,41.0,59.000000,25.0,57.000000,0.000000,15.000000,45.00,22.0,40.00,61.00
7,26.000000,41.000000,28.0,46.000000,12.0,42.000000,15.000000,0.000000,30.00,37.0,25.00,46.00
8,48.000000,42.000000,20.0,24.000000,24.0,12.000000,45.000000,30.000000,0.00,38.0,19.00,19.00
9,33.000000,48.000000,58.0,62.000000,47.0,50.000000,22.000000,37.000000,38.00,0.0,19.00,40.00
